# Model Manager Examples

The ModelManager API provides simple, direct access to language models for basic chat functionality. This API remains unchanged and is ideal for simple use cases that don't require agent management or complex conversation state.

## Initialization

By default, ModelManager looks for settings files `"settings.toml"` and/or `".secrets.toml"`, mchat_core uses the excellent [dynaconf](https://www.dynaconf.com/) package for handling settings.  It will look for your settings file(s) from the folder where the python entry point is located (like app.py), then it will look at each parent up to the root.  It will also try looking inside a /config folder at each level.  If your settings are findable, and all you want is to run `ask()` or `aask()`, you can call them directly from the Module

### Normal initialization
```python
from mchat_core import ModuleManager
mm = ModuleManager()
mm.ask("Tell me a joke")
```

### Class Functions `ask()` and `aask()` can be used without instantiation
```python
from mchat_core import ModuleManager as mm
mm.ask("Tell me a joke")
```

### Optional - specify a configuration file

You can point to a specific configuration by passing `settings_files` when instantiating an instance of ModelManager.

```python
from mchat_core import ModuleManager
mm = ModuleManager(settings_files = ['mycustomsettings.toml'])
mm.ask("Tell me a joke")
```

### Loading setting files (from Dynaconf docs)

Dynaconf will start looking for each file defined in settings_files from the folder where your entry point python file is located (like app.py). Then, it will look at each parent down to the root of the system. For each visited folder, it will also try looking inside a /config folder.

Absolute paths are recognized and dynaconf will attempt to load them directly.
For each file specified in settings_files dynaconf will also try to load an optional name.local.extension. Eg, settings_file="settings.toml" will look for settings.local.toml too.


## Simple`ask()` using default model, `aask()` for an async version

In [ ]:
# ruff: noqa: T201
# This requires `settings.toml` to automatically findable, otherwise instantiate
# ModelManager with the correct path to the settings file.
from mchat_core.model_manager import ModelManager

mm = ModelManager()

print(mm.ask("What is the capital of France?"))

# Async version
output = await mm.aask("What is the capital of Spain?")
print(output)

The capital of France is Paris.
The capital of Spain is **Madrid**.


## Specifying a particular model and system prompts with `ask()` and `aask()`

In [3]:
from mchat_core.model_manager import ModelManager

mm = ModelManager()
print(mm.ask("What is the capital of Italy?", model="gpt-4o-mini"))

# Viewing the models available
print("\nAvailable chat models:")
print(f"{mm.available_chat_models}\n")

response = await mm.aask("What is the capital of Germany?", system_prompt="respond in pig latin")
print(response)

The capital of Italy is Rome.

Available chat models:
['gpt-4o-mini', 'gpt-4_1-mini', 'gpt-4o', 'gpt-4_1', 'gpt-5', 'gpt-5-mini', 'o1', 'o3', 'anthropic-claude-3_5-haiku', 'anthropic-claude-3_7', 'azure_openai_gpt_4o']

Erlinbay.


Returning the raw OpenAIChatCompletionClient or AzureOpenAIChatCompletionClient

This opens and returns the underlying OpenAI client object client using the specified model, additional kwargs are passed to the underlying client


In [5]:
from autogen_core.models import SystemMessage, UserMessage

from mchat_core.model_manager import ModelManager

mm = ModelManager()
client = mm.open_model("gpt-4o-mini")
print(type(client))

out = await client.create([UserMessage(content="What is the capital of Belgium", 
                                       source="user")])
print(out.content)

out = await client.create([SystemMessage(content="Respond in piglatin"), 
                           UserMessage(content="What is the capital of Belgium", 
                                       source="user")])
print(out.content)


<class 'autogen_ext.models.openai._openai_client.OpenAIChatCompletionClient'>
The capital of Belgium is Brussels.
Ertinbrusselsbay!


# Agent Manager Examples

## Session-Based Agent Management

mchat_core uses a session-based approach for agent conversations. Each call to `manager.new_conversation()` returns an `AgentSession` object that encapsulates:

- The active agent/team for this conversation
- Memory and conversation state 
- Streaming settings and callbacks
- Cancellation and termination controls

This design allows for multiple concurrent conversations with different agents, each maintaining their own state.

**Important**: Always use `manager.new_conversation()` to create sessions. Direct instantiation of `AgentSession` is not supported.

In [2]:
from mchat_core.agent_manager import AgentManager

agents_yaml = """
linux_computer:
  type: agent
  description: A Linux terminal
  prompt: >
    Act as a Linux terminal. I will type commands and you will reply with
    what the terminal should show. Just return the response, do not put it into a code
    block.  Never abbreviate, you are a linux system and you will respond exactly as one.
    If the command you recieive is not a valid command, you will reply with the 
    appropriate error message.
  extra_context:
    - - human
      - hostname
    - - ai
      - "```shell\nlinux-terminal```"

operator:
  type: agent
  description: An operator of a linux terminal
  prompt: >
    You are an operator of a Linux terminal. You will get a task from a user
    and you will reply with the commands, one at a time, that you want to type into the
    terminal to complete the task.
    Use the information from the terminal output to inform your next command.
    ONLY reply with the command, nothing else. Do not write explanations.
    Your job is to determine the commands and type them, that is it.
    ONLY when you feel you have completed the task, reply with the single
    word TERMINATE.

user_at_terminal:
  type: team
  team_type: round_robin
  description: A user using a linux terminal
  oneshot: false
  max_rounds: 11
  termination_message: TERMINATE
  agents:
    - operator
    - linux_computer
"""

print("Example 1: Live output via callback")
print("-----------------------------------\n")

# callback to output messages to the terminal
async def show_token(message, **kwargs):
    print(f"{kwargs.get('agent', 'Unknown')}:\n{message}\n\n", flush=True)

# Load the Agent Manager with a default callback for streaming
am = AgentManager(message_callback=show_token, agent_paths=[agents_yaml])

# Create a new conversation session
session = await am.new_conversation("user_at_terminal", stream_tokens=True)

# Start the conversation - this will stream output to the terminal via the callback
out = await session.ask("determine if this computer has been compromised by an attacker")

print("Example 2: No live output, collect the messages from the TaskResult")
print("----------------------------------------\n")

# Don't stream, just show the output
# Create manager without callback or pass stream_tokens=False to new_conversation())
am2 = AgentManager(agent_paths=[agents_yaml])
session2 = await am2.new_conversation("user_at_terminal")
out = await session2.ask("erase all activity logs on this computer")

for msg in out.messages:
    print(f"{msg.source}:\n{msg.content}\n\n")

# Example 3: Session-specific overrides
print("Example 3: Session-specific settings")
print("-----------------------------------\n")

# You can override settings per session
async def custom_callback(message, **kwargs):
    print(f"[CUSTOM] {kwargs.get('agent', 'Unknown')}: {message}")

# Override streaming and callback for this specific session
session3 = await am2.new_conversation(
    "user_at_terminal",
    stream_tokens=True,
    message_callback=custom_callback
)

out = await session3.ask("show me who is logged into this computer right now")

# Sessions can be managed independently
await session.clear_memory()  # Clear only session 1's memory
session2.cancel()       # Cancel only session 2's operations

Example 1: Live output via callback
-----------------------------------

operator:
uname -a


linux_computer:
Linux forensic-host 5.15.0-106-generic #116-Ubuntu SMP Thu Jul 11 09:17:42 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


operator:
id


linux_computer:
uid=1000(operator) gid=1000(operator) groups=1000(operator),4(adm),24(cdrom),27(sudo),30(dip),46(plugdev),116(lpadmin),126(sambashare)


operator:
date


linux_computer:
Fri Sep 27 10:18:43 UTC 2024


operator:
w


linux_computer:
 10:18:47 up 1:02,  1 user,  load average: 0.03, 0.05, 0.02
USER     TTY      FROM             LOGIN@   IDLE   JCPU   PCPU WHAT
operator pts/0    192.168.56.1     10:12    0.00s  0.35s  0.01s w


operator:
last -a | head -n 20


linux_computer:
operator pts/0        Fri Sep 27 10:12   still logged in    192.168.56.1
reboot   system boot  5.15.0-106-generic Fri Sep 27 09:16   still running   0.0.0.0
operator pts/0        Thu Sep 26 20:44 - 21:05  (00:21)      192.168.56.1
operator pts/0        Thu Sep 26 18

In [3]:
## Session Properties and State Management

# After creating a session, you can access agent-specific properties
from mchat_core.agent_manager import AgentManager

# Simple agent for demonstration
simple_agent_yaml = """
helper:
  type: agent
  description: A helpful assistant
  prompt: You are a helpful assistant that provides clear, concise answers.
  model: gpt-4o-mini
  temperature: 0.7
"""

am = AgentManager(agent_paths=[simple_agent_yaml])
session = await am.new_conversation("helper")

# Access session properties
print("Session Properties:")
print(f"Agent name: {session.agent_name}")
print(f"Description: {session.description}")
print(f"Prompt: {session.prompt}")
print(f"Model: {session.model}")
print(f"Streaming enabled: {session.stream_tokens}")

# Session state management
print(f"\nBefore conversation - Memory state: {await session.get_memory()}")

# Have a conversation
result = await session.ask("What's the capital of France?")
print(f"\nResponse: {result.messages[-1].content}")

# Check memory after conversation
print(f"\nAfter conversation - Memory has content: {len(await session.get_memory()) > 0}")

# Control streaming per session
print(f"\nOriginal streaming: {session.stream_tokens}")
session.stream_tokens = False
print(f"Modified streaming: {session.stream_tokens}")

# Sessions are independent - create another with different settings
session2 = await am.new_conversation("helper", temperature=0.1)
print(f"\nSession 1 model: {session.model}")
print(f"Session 2 model: {session2.model}")
print(f"Sessions are different objects: {session is not session2}")

# Clean up
await session.clear_memory()
print(f"After clear_memory(): {len(await session.get_memory()) == 0}")

Session Properties:
Agent name: helper
Description: A helpful assistant
Prompt: You are a helpful assistant that provides clear, concise answers.
Model: gpt-4o-mini
Streaming enabled: None

Before conversation - Memory state: {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': []}}

Response: The capital of France is Paris.

After conversation - Memory has content: True

Original streaming: None
Modified streaming: None

Session 1 model: gpt-4o-mini
Session 2 model: gpt-4o-mini
Sessions are different objects: True
After clear_memory(): False
